In [ ]:
import os
import difflib
import collections
from datetime import datetime
import openpyxl

In [ ]:
# enter basic informations like checkpoint_path and configs_path,
ckpt_path = "/root/ext_data/studiogan_ckpt/CIFAR10"
cfg_path = "/root/code/joong_test/PyTorch-StudioGAN/src/configs/CIFAR10"
dataset_path = "/root/code/joong_test/data/CIFAR10"
dataset = "CIFAR10"
num_eval = 1

ckpt_list = os.listdir(ckpt_path)
cfg_list = os.listdir(cfg_path)

# load Excel File and read informations
wb = openpyxl.load_workbook("CIFAR10.xlsx")
ws = wb['Sheet1']

runs = collections.defaultdict(lambda : collections.defaultdict(str))

all_clumns = ws.columns
for idx,col in enumerate(all_clumns):
    if idx >=2 and col[1].value != None:
        assert col[6].value in ckpt_list, str(col[6].value) + " Not in ckpt_list"
        assert col[7].value in ckpt_list, str(col[7].value) + " Not in ckpt_list"
        assert col[8].value in ckpt_list, str(col[8].value) + " Not in ckpt_list"
        runs[col[1].value]["run_1"] = col[6].value
        runs[col[1].value]["run_2"] = col[7].value
        runs[col[1].value]["run_3"] = col[8].value
        if col[1].value == "ACGAN":
            runs[col[1].value]["cfg"] = difflib.get_close_matches(col[1].value+"-Mod", cfg_list, n=1, cutoff=0)[0]
        else:
            runs[col[1].value]["cfg"] = difflib.get_close_matches(col[1].value, cfg_list, n=1, cutoff=0)[0]
        print(f"{col[1].value} matched with {runs[col[1].value]['cfg']}")

In [ ]:
# Check if evey excel column is matched with proper configurations and modify config if needed.
runs["TACGAN"]["cfg"] = "ACGAN-Mod-TAC.yaml"

print("After fixing...")
for key, value in runs.items():
    print(f"{key} matched with {value['cfg']}")

In [ ]:
# write script file
i = 1
with open("eval_clean_"+dataset+".sh", "w") as f:
    f.write("wandb offline\n")
    prefix_style = "CUDA_VISIBLE_DEVICES=0 python src/main.py -metrics is fid prdc" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
    if dataset == "ImageNet":
        prefix_ = "CUDA_VISIBLE_DEVICES=0 python src/main.py -std_stat -std_max 256 -std_step 256 -metrics is fid prdc" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        valid = " -ref valid"
    else:
        prefix_ = "CUDA_VISIBLE_DEVICES=0 python src/main.py -std_stat -std_max 128 -std_step 128 -metrics is fid prdc" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        valid = " -ref test"
    train, legacy, clean = " -ref train", " --resize_fn legacy", " --resize_fn clean"
    for (key, value) in runs.items():
        others1 = " -ckpt " + os.path.join(ckpt_path, value['run_1']) + " -cfg " + os.path.join(cfg_path, value['cfg']) + " \n"
        others2 = " -ckpt " + os.path.join(ckpt_path, value['run_2']) + " -cfg " + os.path.join(cfg_path, value['cfg']) + " \n"
        others3 = " -ckpt " + os.path.join(ckpt_path, value['run_3']) + " -cfg " + os.path.join(cfg_path, value['cfg']) + " \n"
        if "Style" in key:
            prefix = prefix_style
        else:
            prefix = prefix_
        f.write(prefix+train+clean+others1)
        f.write(f"echo =================={i}/{3*len(runs)} completed =============\n")
        i+=1 
        f.write(prefix+train+clean+others2)
        f.write(f"echo =================={i}/{3*len(runs)} completed =============\n")
        i+=1
        f.write(prefix+train+clean+others3)
        f.write(f"echo =================={i}/{3*len(runs)} completed =============\n")
        i+=1
    

i = 1    
with open("eval_legacy_"+dataset+".sh", "w") as f:
    f.write("wandb offline\n")
    prefix_style = "CUDA_VISIBLE_DEVICES=0 python src/main.py -metrics is fid prdc" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
    if dataset == "ImageNet":
        prefix_ = "CUDA_VISIBLE_DEVICES=0 python src/main.py -std_stat -std_max 256 -std_step 256 -metrics is fid prdc" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        valid = " -ref valid"
    else:
        prefix_ = "CUDA_VISIBLE_DEVICES=0 python src/main.py -std_stat -std_max 128 -std_step 128 -metrics is fid prdc" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        valid = " -ref test"
    train, legacy, clean = " -ref train", " --resize_fn legacy", " --resize_fn clean"
    for (key, value) in runs.items():
        others1 = " -ckpt " + os.path.join(ckpt_path, value['run_1']) + " -cfg " + os.path.join(cfg_path, value['cfg']) + " \n"
        others2 = " -ckpt " + os.path.join(ckpt_path, value['run_2']) + " -cfg " + os.path.join(cfg_path, value['cfg']) + " \n"
        others3 = " -ckpt " + os.path.join(ckpt_path, value['run_3']) + " -cfg " + os.path.join(cfg_path, value['cfg']) + " \n"
        if "Style" in key:
            prefix = prefix_style
        else:
            prefix = prefix_
        f.write(prefix+train+legacy+others1)
        f.write(f"echo =================={i}/{3*len(runs)} completed =============\n")
        i+=1 
        f.write(prefix+train+legacy+others2)
        f.write(f"echo =================={i}/{3*len(runs)} completed =============\n")
        i+=1
        f.write(prefix+train+legacy+others3)
        f.write(f"echo =================={i}/{3*len(runs)} completed =============\n")
        i+=1


In [ ]:
#### RUN 
#### bash eval_clean_CIFAR10.sh
#### bash eval....sh